In [ ]:
from openai import OpenAI

def generate_test_code(
    fqn: str,
    signature: str,
    jimple: str,
    callees: str,
    method_modifiers: str,
    class_modifiers: str,
    method_javadoc: str,
    method_body: str,
    imports: str,
    class_javadoc: str,
    class_fnc: str
) -> str:
    """
    Generates a complete and verbose Java unit test using OpenAI based on detailed information about a focal method.

    Returns:
        str: The generated Java test code as a string.
    """

    client = OpenAI(api_key="not-telling-you")

    prompt = f"""
**Test Generator {{

@persona {{
You are a Java expert and a professional test engineer.
Your role is to write high-coverage, verbose, and well-structured JUnit 4 test code for a **single Java method** under test.
Your output must consist of code only — no explanation or commentary. Output a complete and compilable Java test file from package declaration to the final closing brace.
}}

@terminology {{
- focal_method_fqn: The fully qualified name of the class that contains the method under test.
- focal_method_signature: The declaration/signature of the method to be tested.
- focal_method_jimple: The Jimple representation of the focal method, showing low-level control and data flow.
- focal_method_body: The method's original Java source code.
- focal_method_modifiers: Access and other modifiers on the focal method (if present).
- focal_class_modifiers: Access and other modifiers on the focal class (if present).
- focal_method_javadoc: JavaDoc comment above the method (if present).
- focal_class_javadoc: JavaDoc comment above the class (if present).
- callee_edges: All edges to other methods that the focal method calls, including their signatures.
- field_and_constructor_declarations: All fields and constructors declared in the focal class.
- import_statements: All necessary import statements found in the focal class file (excluding test imports).
}}

@instruction {{

@command: Using ALL available information, generate a verbose and high-coverage unit test for the provided focal method using JUnit 4.
The test should aim to exercise as many execution paths as possible, leveraging the Jimple code, class/method modifiers, method calls, JavaDoc, and class context (fields & constructors).
If exception or edge-case paths are possible per Jimple/control flow, include `try-catch` blocks.
If the method or containing class is abstract, provide a concrete implementation as required.
Always consider edge cases: e.g., nulls, empty strings/arrays/collections, boundary values, and large or unusual inputs.
Make use of JavaDoc (when available) to reason about preconditions, expectations, and side effects.

@rule1: Output only the Java test code — no explanation, no markdown formatting, no commentary.
@rule2: Use JUnit 4 only. All test methods must include `@Test` and use `org.junit.Test` and `static org.junit.Assert.*` at the minimum.
@rule3: If the focal_method_body has many if-else branches, create a separate test method/case for each branch to ensure high coverage.
@rule4: Also include any additional import statements provided in the import_statements section.
@rule5: Do not include or modify the focal method source in the test file.
@rule6: If the focal method is private or within a private/inner class, use Java Reflection to access and test it.
@rule7: If the focal method throws exceptions, catch them in `try-catch` blocks. Do not use `assertThrows`.
@rule8: Name the test class in this format: <ClassName>_<MethodName>_<MethodParameterTypes>_Test.
@rule9: The test class must be a valid standalone Java file — include package declaration if provided in the focal_method_fqn.
@rule10: When useful, derive test object instantiations from the field_constructor_declarations, or callee_edges.
@rule11: If some field, constructor, or callee information is missing, adapt gracefully to what is available.
@rule12: Each test method must cover a unique logical path or edge case. Prefer more tests with a variety of inputs over fewer.
}}

@format {{

@input:
###focal_method_fqn
{fqn}

###focal_method_signature
{signature}

###focal_method_jimple
{jimple}

###focal_method_body
{method_body}

###focal_method_modifiers
{method_modifiers}

###focal_class_modifiers
{class_modifiers}

###focal_method_javadoc
{method_javadoc}

###focal_class_javadoc
{class_javadoc}

###callee_edges
{callees}

###field_and_constructor_declarations
{class_fnc}

###import_statements
{imports}

@output:
###generated_test_code
}}
}}"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=16000
    )

    return response.choices[0].message.content

In [41]:
import pandas as pd

# Define FQN prefixes
cr_fqn = "org.apache.commons.lang3.CharRange"
ctu_fqn = "org.apache.commons.lang3.CharSetUtils"
wu_fqn = "org.apache.commons.lang3.text.WordUtils"
ai_fqn = "org.apache.commons.lang3.concurrent.AtomicInitializer"
cpt_fqn = "org.apache.commons.lang3.text.translate.CodePointTranslator"

# Load the original CSV
df = pd.read_csv("/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data.csv")

# Apply all filters:
filtered_df = df[
    df['FQN'].str.startswith((cr_fqn, ctu_fqn, wu_fqn, ai_fqn, cpt_fqn)) &           # FQN filter
    ~df['Signature'].str.contains("<init>") &                                        # Exclude constructors
    df['Imports'].notna()                                                            # Exclude null or empty Imports
].copy()

# Save the filtered DataFrame
filtered_df.to_csv("/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data_Task4-1.csv", index=False)

In [42]:
# Load the filtered dataset
df = pd.read_csv("/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data_Task4-1.csv")

# Sample 3 random methods
samples = df.sample(n=3, random_state=42)  # Reproducible

# Loop over sampled rows
for index, row in samples.iterrows():
    fqn = row['FQN']
    signature = row['Signature']
    jimple = row['Jimple']
    callees = row['Callees']
    method_modifiers = row['MethodModifiers']
    class_modifiers = row['ClassModifiers']
    method_javadoc = row['JavaDoc']
    method_body = row['MethodBody']
    imports = row['Imports']
    class_javadoc = row['classJavaDoc']
    class_fnc = row['classFnCs']

    # Generate test using all 11 fields
    test_code = generate_test_code(
        fqn=fqn,
        signature=signature,
        jimple=jimple,
        callees=callees,
        method_modifiers=method_modifiers,
        class_modifiers=class_modifiers,
        method_javadoc=method_javadoc,
        method_body=method_body,
        imports=imports,
        class_javadoc=class_javadoc,
        class_fnc=class_fnc
    )

    # Print the output
    print(f"==== Test for {fqn} ====")
    print("Generated Test Code:")
    print(test_code)
    print("\n====================\n")


==== Test for org.apache.commons.lang3.CharRange.toString() ====
Generated Test Code:
```java
package org.apache.commons.lang3;

import org.junit.Test;
import static org.junit.Assert.*;

public class CharRange_toString__Test {

    @Test
    public void testToString_NonNegatedSingleCharacter() {
        CharRange charRange = createCharRange('a', 'a', false);
        String result = charRange.toString();
        assertEquals("a", result);
    }

    @Test
    public void testToString_NonNegatedRange() {
        CharRange charRange = createCharRange('a', 'z', false);
        String result = charRange.toString();
        assertEquals("a-z", result);
    }

    @Test
    public void testToString_NegatedSingleCharacter() {
        CharRange charRange = createCharRange('a', 'a', true);
        String result = charRange.toString();
        assertEquals("^a", result);
    }

    @Test
    public void testToString_NegatedRange() {
        CharRange charRange = createCharRange('a', 'z', true);
 